In [1]:
import sys
sys.path.append("../")

In [14]:
import numpy as np
import torch
import os
import post_eval
import dataset
from monai.networks.layers import Norm
from monai.networks.nets import (
    UNet
)
device = "cuda"
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import DataLoader
import albumentations as A

In [15]:
%matplotlib inline
import matplotlib.pyplot as plt

In [16]:
PostEvaluationImageGenertor = post_eval.PostEvaluationImageGenertor
OctDataset = dataset.OctDataset

In [72]:
model_dir = "../models/BasicUnet_NoAug_6layers/Model_2022-05-01 15:08:08.139250"
weight_file = "best_metric_model_weights.pth"
output_dir = "./6layer_NoAug_Unet"
base_data_dir = "../data/full_window"

In [73]:
train_data_image_dir = os.path.join(base_data_dir, "train_images")
train_data_label_dir = os.path.join(base_data_dir, "train_labels")
val_data_image_dir   = os.path.join(base_data_dir, "val_images")
val_data_label_dir   = os.path.join(base_data_dir, "val_labels")
test_data_image_dir  = os.path.join(base_data_dir, "test_images")
test_data_label_dir  = os.path.join(base_data_dir, "test_labels")

assert(os.path.exists(train_data_image_dir))
assert(os.path.exists(train_data_label_dir))
assert(os.path.exists(val_data_image_dir))
assert(os.path.exists(val_data_label_dir))
assert(os.path.exists(test_data_image_dir))
assert(os.path.exists(test_data_label_dir))

In [74]:
weight_path = os.path.join(model_dir, weight_file)
assert(os.path.exists(output_dir))
assert(os.path.exists(weight_path))

In [75]:
val_evaluator = PostEvaluationImageGenertor(desc="Validation")
test_evaluator = PostEvaluationImageGenertor(desc="Test")

In [76]:
#### Needed To Get Labels in [1, H, W] For Training Loop
def AddLabelChannel(labels, **params):
    return labels.unsqueeze(dim=0)

t = A.Compose([
    ToTensorV2(),
    A.Lambda(name="Add Channel Dimension",
             mask=AddLabelChannel,
             always_apply=True)
])

In [77]:
train_ds = OctDataset(train_data_image_dir,
                        train_data_label_dir,
                          sort=True,
                          transforms=t,
                          num_samples=-1)
val_ds   = OctDataset(val_data_image_dir,
                          val_data_label_dir,
                          sort=True,
                          transforms=t,
                          num_samples=-1)
test_ds  = OctDataset(test_data_image_dir,
                          test_data_label_dir,
                          sort=True,
                          transforms=t,
                          num_samples=-1)

In [78]:
train_loader = DataLoader(
        train_ds,
        batch_size=16,
        num_workers=0,
        pin_memory=True,
        shuffle=False)

val_loader = DataLoader(
        val_ds,
        batch_size=16,
        num_workers=0,
        pin_memory=True,
        shuffle=False)

test_loader = DataLoader(
        test_ds,
        batch_size=16,
        num_workers=0,
        pin_memory=True,
        shuffle=False)

model= UNet(
            spatial_dims=2,
            in_channels=1,
            out_channels=3,
            channels=[8, 16, 32, 48, 96, 192, 384],
            strides=[2, 2, 2, 2, 2, 2],
            norm=Norm.BATCH,
            dropout=.1
).to(device)
model.load_state_dict(torch.load(weight_path))

model= UNet(
            spatial_dims=2,
            in_channels=1,
            out_channels=3,
            channels=[16, 32, 48, 96],
            strides=[2, 2, 2],
            norm=Norm.BATCH,
            dropout=.1
).to(device)
model.load_state_dict(torch.load(weight_path))

model= UNet(
            spatial_dims=2,
            in_channels=1,
            out_channels=3,
            channels=[16, 32, 48, 96, 192],
            strides=[2, 2, 2, 1],
            norm=Norm.BATCH,
            dropout=0.2
).to(device)
model.load_state_dict(torch.load(weight_path))

In [79]:
model= UNet(
            spatial_dims=2,
            in_channels=1,
            out_channels=3,
            channels=[16, 32, 48, 96, 192, 384],
            strides=[3, 3, 2, 1, 1],
            norm=Norm.BATCH,
            dropout=0.1
).to(device)
model.load_state_dict(torch.load(weight_path))

<All keys matched successfully>

In [80]:
test_evaluator(model, test_loader, output_dir)

==== Evaluation Test ====
Running standard pointwise prediction...


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 57 but got size 58 for tensor number 1 in the list.

In [60]:
val_evaluator(model, val_loader, output_dir)

==== Evaluation Validation ====
Running standard pointwise prediction...
Creating Visual For Instance Validation-1
Creating Visual For Instance Validation-2
Creating Visual For Instance Validation-3
Creating Visual For Instance Validation-4
Creating Visual For Instance Validation-5
Creating Visual For Instance Validation-6
Creating Visual For Instance Validation-7
Creating Visual For Instance Validation-8
Creating Visual For Instance Validation-9
Creating Visual For Instance Validation-10
Creating Visual For Instance Validation-11
Creating Visual For Instance Validation-12
Creating Visual For Instance Validation-13
Creating Visual For Instance Validation-14
Creating Visual For Instance Validation-15
Creating Visual For Instance Validation-16
Creating Visual For Instance Validation-17
Creating Visual For Instance Validation-18
Creating Visual For Instance Validation-19
Creating Visual For Instance Validation-20
Creating Visual For Instance Validation-21
Creating Visual For Instance Vali

tensor(0.7314, device='cuda:0')